In [ ]:
# Library

import torch
import math
import re
import numpy as np
import pandas as pd
from torch.distributions import Laplace

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForCausalLM
from transformers import GPT2LMHeadModel
from transformers import AutoModelForSequenceClassification

from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from scipy.stats import vonmises_fisher
import torch.nn.functional as F

from typing import Dict, List, Optional

from openai import OpenAI
import glob

from collections import Counter

from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Run once per notebook
import shutil, matplotlib as mpl

USE_TEX = shutil.which("latex") is not None   # auto-detect; False on Colab by default
mpl.rcParams.update({"text.usetex": USE_TEX, "axes.unicode_minus": False})

if USE_TEX:
    # Real LaTeX path (if you *do* have TeX available)
    mpl.rcParams.update({
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman", "CMU Serif", "Times New Roman", "DejaVu Serif"],
        "text.latex.preamble": r"\usepackage{amsmath}\usepackage{bm}\usepackage{siunitx}"
    })
else:
    # LaTeX-like look without LaTeX installed
    mpl.rcParams.update({
        "text.usetex": False,
        # Use a LaTeX-y serif + STIX math (good match for LaTeX/Times);
        # if you prefer Computer Modern look, change 'stix' -> 'cm'
        "font.family": "serif",
        "font.serif": ["STIX Two Text", "STIXGeneral", "DejaVu Serif", "Times New Roman"],
        "mathtext.fontset": "stix",
        "mathtext.rm": "serif",
        "mathtext.it": "serif:italic",
        "mathtext.bf": "serif:bold",
    })

In [ ]:
client = OpenAI(api_key="Your_API_Key")  # needs OPENAI_API_KEY


In [ ]:
# --- Load tokenizer and GPT-2 model ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", output_hidden_states=True)
embedding_table = model.get_input_embeddings().weight.detach()
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.eval()

# Base tokenizer pad fix (optional)
if getattr(tokenizer, "pad_token_id", None) is None and getattr(tokenizer, "eos_token", None) is not None:
    tokenizer.pad_token = tokenizer.eos_token

# Load a light GPT-2 model
gpt2_tok = AutoTokenizer.from_pretrained("distilgpt2")
if gpt2_tok.pad_token is None:
    gpt2_tok.pad_token = gpt2_tok.eos_token
gpt2_model = AutoModelForCausalLM.from_pretrained("distilgpt2").to(
    "cuda" if torch.cuda.is_available() else "cpu"
).eval()

# --- Extract embedding table ---
# Normalize embedding table for search
norm_embedding_table = torch.nn.functional.normalize(embedding_table, dim=1)

In [ ]:
# --- load all STAMP_Polar and STAMP_Laplace datasets ---
csv_files_polar_inf = glob.glob("/content/yelp_inf_polar_sweep_avg_epsilon_*.csv") # Changed to yelp
csv_files_laplace_inf = glob.glob("/content/yelp_inf_laplace_sweep_avg_epsilon_*.csv") # Changed to yelp

csv_files = csv_files_polar_inf + csv_files_laplace_inf
print("Files found by glob:", csv_files)

dataframes = {}

for file in csv_files:
    df_name = file.replace(".csv", "")
    dataframes[df_name] = pd.read_csv(file)

print("Number of dataframes loaded:", len(dataframes))

In [ ]:
# Get the answers

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        return re.sub(r'[^\w\s]', '', text)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()

def compute_f1(prediction, truth):
    pred_tokens = get_tokens(prediction)
    truth_tokens = get_tokens(truth)

    # if either is empty, return 1 if both are empty, 0 otherwise
    if not pred_tokens and not truth_tokens:
        return 1.0
    if not pred_tokens or not truth_tokens:
        return 0.0

    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    num_common = sum(common_tokens.values())

    if num_common == 0:
        return 0.0

    precision = num_common / len(pred_tokens)
    recall = num_common / len(truth_tokens)

    return (2 * precision * recall) / (precision + recall)

def evaluate_answer(predicted_answer: str, true_answers: list[str]):
    """
    Evaluates a predicted answer against a list of true answers using F1 score and exact match.

    Args:
        predicted_answer: The generated answer string.
        true_answers: A list of true answer strings.

    Returns:
        A tuple containing:
            - The maximum F1 score achieved against any of the true answers.
            - A boolean indicating whether an exact match was found against any of the true answers.
    """
    max_f1 = 0.0
    exact_match = False

    normalized_prediction = normalize_answer(predicted_answer)

    for true_answer in true_answers:
        normalized_true = normalize_answer(true_answer)

        # Exact Match
        if normalized_prediction == normalized_true:
            exact_match = True

        # F1 Score
        f1 = compute_f1(predicted_answer, true_answer)
        max_f1 = max(max_f1, f1)

    return max_f1, exact_match


def calculate_cosine_similarity(prediction: str, truths: list[str], model) -> float:
    """
    Calculates the cosine similarity between the prediction and each true answer
    using Sentence-BERT embeddings and returns the maximum similarity.
    """
    if not prediction or not truths:
        return 0.0

    # Encode the prediction
    prediction_embedding = model.encode(prediction, convert_to_tensor=True)

    max_similarity = 0.0
    for truth in truths:
        if not truth:
            continue
        # Encode the true answer
        truth_embedding = model.encode(truth, convert_to_tensor=True)

        # Calculate cosine similarity
        similarity = util.pytorch_cos_sim(prediction_embedding, truth_embedding).item()
        max_similarity = max(max_similarity, similarity)

    return max_similarity


In [ ]:
# Initialize Sentence Transformer model
model_sentence = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:

# Load kmack Yelp classifier
tokenizer = AutoTokenizer.from_pretrained("kmack/YELP-Review_Classifier")
model = AutoModelForSequenceClassification.from_pretrained("kmack/YELP-Review_Classifier")

num_rounds = 3
all_results = []

for round_num in range(num_rounds):
    print(f"--- Starting Round {round_num + 1}/{num_rounds} ---")
    results_list = []

    for df_name, df in dataframes.items():
        print(f"Processing DataFrame: {df_name}")
        for index, row in df.iterrows():
            question = row["question"]
            true_answers = [str(row["rating"])]  # Assuming 'rating' column holds gold label
            privatized_context = row["privatized_context"]
            original_context = row["original_context"]

            try:
                # Encode privatized context
                inputs = tokenizer(privatized_context, return_tensors="pt", truncation=True, padding=True)

                # Run classification
                with torch.no_grad():
                    outputs = model(**inputs)
                predicted_label = torch.argmax(outputs.logits, dim=1).item()

                # Store as string for comparison
                answer_privatized = str(predicted_label)

            except Exception as e:
                print(f"Error running classifier: {e}")
                answer_privatized = ""

            # Evaluate answer
            f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)

            results_list.append({
                "dataframe": df_name,
                "question": question,
                "true_answers": true_answers,
                "original_context": original_context,
                "privatized_context": privatized_context,
                "answer_privatized": answer_privatized,
                "f1_privatized": f1_privatized,
                "em_privatized": em_privatized,
            })

    df_round_results = pd.DataFrame(results_list)
    all_results.append(df_round_results)

print(f"\nFinished {num_rounds} rounds of evaluation.")


In [ ]:
# Concatenate all results DataFrames into a single DataFrame
df_all_results_combined = pd.concat(all_results, ignore_index=True)

# Save the combined DataFrame to a CSV file
csv_output_path = "Yelp_evaluation_results_inf_rounds.csv"
df_all_results_combined.to_csv(csv_output_path, index=False)

print(f"All evaluation results saved to {csv_output_path}")

In [ ]:
# Load the Yelp GPT-4 Fill dataset
file_path_yelp_gpt4_fill = "/content/yelp_gpt4_fill_tau_0.5.csv" # Assuming this is the correct filename based on available files
try:
    df_yelp_gpt4_fill = pd.read_csv(file_path_yelp_gpt4_fill)
    print(f"Loaded dataset: {file_path_yelp_gpt4_fill}")
    display(df_yelp_gpt4_fill.head())
except FileNotFoundError:
    print(f"Error: File not found at {file_path_yelp_gpt4_fill}")
    df_yelp_gpt4_fill = None

In [ ]:
# Load the kmack Yelp Review Classifier
tokenizer = AutoTokenizer.from_pretrained("kmack/YELP-Review_Classifier")
model = AutoModelForSequenceClassification.from_pretrained("kmack/YELP-Review_Classifier")

if df_yelp_gpt4_fill is not None:
    results_list_yelp_gpt4_fill = []

    for index, row in df_yelp_gpt4_fill.iterrows():
        question = row["question"]  # Not used by classifier
        true_rating_original = row["rating"]  # 0–4 rating scale
        true_answers = [str(true_rating_original)]  # for evaluate_answer()

        privatized_context = row["privatized_context_gpt4_fill"]

        try:
            # Encode privatized context
            inputs = tokenizer(privatized_context, return_tensors="pt", truncation=True, padding=True)

            # Run classification
            with torch.no_grad():
                outputs = model(**inputs)
            predicted_label = torch.argmax(outputs.logits, dim=1).item()

            # Map to string
            answer_privatized = str(predicted_label)

        except Exception as e:
            print(f"Error running kmack/YELP-Review_Classifier on row {index}: {e}")
            answer_privatized = ""

        # Evaluate predicted label vs. true rating
        f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)

        results_list_yelp_gpt4_fill.append({
            "question": question,
            "true_rating": true_rating_original,
            "true_answers": true_answers,
            "privatized_context": privatized_context,
            "predicted_sentiment_label": answer_privatized,
            "f1_privatized": f1_privatized,
            "em_privatized": em_privatized,
        })

    # Convert results list to DataFrame
    df_yelp_gpt4_fill_results = pd.DataFrame(results_list_yelp_gpt4_fill)
    display(df_yelp_gpt4_fill_results.head())

    # Compute average scores
    average_scores_yelp_gpt4_fill = {
        'mean_f1_privatized_kmack': df_yelp_gpt4_fill_results['f1_privatized'].mean(),
        'mean_em_privatized_kmack': df_yelp_gpt4_fill_results['em_privatized'].mean(),
    }
    df_average_scores_yelp_gpt4_fill = pd.DataFrame([average_scores_yelp_gpt4_fill])
    display(df_average_scores_yelp_gpt4_fill)

    # Save results
    csv_output_path_yelp_gpt4_fill = "yelp_gpt4_fill_kmack_results.csv"
    df_yelp_gpt4_fill_results.to_csv(csv_output_path_yelp_gpt4_fill, index=False)

    print(f"Evaluation results for Yelp GPT-4 Fill with kmack/YELP-Review_Classifier saved to {csv_output_path_yelp_gpt4_fill}")


In [ ]:
# Load all CSV files in /content/

csv_files = glob.glob("/content/*.csv")
print(f"Found {len(csv_files)} CSV files in /content/: {csv_files}")

dataframes = {}
for file in csv_files:
    try:
        df_name = file.replace("/content/", "").replace(".csv", "")
        dataframes[df_name] = pd.read_csv(file)
        print(f"Successfully loaded {file} into dataframe '{df_name}'")
        display(dataframes[df_name].head()) # Display the head of the dataframe
    except Exception as e:
        print(f"Error loading {file}: {e}")

print(f"✅ Number of dataframes loaded: {len(dataframes)}")

In [ ]:
plt.figure(figsize=(12, 7))

# Inf Laplace
df_laplace = df_aggregated_inf_metrics[df_aggregated_inf_metrics['method'] == 'Inf Laplace']
plt.plot(df_laplace['epsilon'], df_laplace['mean_accuracy'],
         label='Inf Laplace', marker='o', linestyle='-')

# Inf Polar
df_polar = df_aggregated_inf_metrics[df_aggregated_inf_metrics['method'] == 'Inf Polar']
plt.plot(df_polar['epsilon'], df_polar['mean_accuracy'],
         label='Inf Polar', marker='o', linestyle='--')

# Baseline (kmack classifier)
if 'average_scores_yelp_gpt4_fill' in globals() and \
   'mean_em_privatized_kmack' in average_scores_yelp_gpt4_fill:
    baseline_acc = average_scores_yelp_gpt4_fill['mean_em_privatized_kmack']
    plt.axhline(y=baseline_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill)')
    print(f"Baseline accuracy (kmack): {baseline_acc:.3f}")
else:
    print("⚠️ Baseline accuracy not found — check variable name!")

plt.title('Accuracy vs. Epsilon')
plt.xlabel('Epsilon')
plt.ylabel('Accuracy (Exact Match)')
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend(title='Method')
plt.show()


In [ ]:
# Process the 'Yelp_evaluation_results_inf_rounds' dataframe
df_inf_results = dataframes['Yelp_evaluation_results_inf_rounds'].copy()

# Extract epsilon from filename
df_inf_results['epsilon'] = df_inf_results['dataframe'].apply(
    lambda x: float(x.split('_')[-1]) # Assuming filenames like ..._200.00, ..._400.00
)

# Add method column (Laplace or Polar)
df_inf_results['method'] = df_inf_results['dataframe'].apply(
    lambda x: 'Inf Laplace' if 'laplace' in x.lower() else 'Inf Polar'
)

# Aggregate mean accuracy by epsilon + method
df_aggregated_inf_metrics = (
    df_inf_results.groupby(['epsilon', 'method'])
    .agg(mean_accuracy=('em_privatized', 'mean'))
    .reset_index()
    .sort_values('epsilon')
)

display(df_aggregated_inf_metrics)

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import matplotlib.pyplot as plt

# --- 1) Non-private Yelp baseline (first 50 test samples) ---
yelp = load_dataset("yelp_review_full", split="test[:50]")

tokenizer = AutoTokenizer.from_pretrained("kmack/YELP-Review_Classifier")
model = AutoModelForSequenceClassification.from_pretrained("kmack/YELP-Review_Classifier")

correct = 0
for example in yelp:
    text = example["text"]
    true_label = example["label"]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    if pred == true_label:
        correct += 1

baseline_yelp50_acc = correct / len(yelp)
print(f"Baseline (Non-Private)): {baseline_yelp50_acc:.3f}")

# --- 2) Inf Laplace & Inf Polar curves ---
plt.figure(figsize=(12, 7))

df_laplace = df_aggregated_inf_metrics[df_aggregated_inf_metrics['method'] == 'Inf Laplace']
plt.plot(df_laplace['epsilon'], df_laplace['mean_accuracy'],
         label='Inf Laplace', marker='o', linestyle='-')

df_polar = df_aggregated_inf_metrics[df_aggregated_inf_metrics['method'] == 'Inf Polar']
plt.plot(df_polar['epsilon'], df_polar['mean_accuracy'],
         label='Inf Polar', marker='o', linestyle='--')

# --- 3) GPT-4 Fill + kmack classifier baseline ---
if 'average_scores_yelp_gpt4_fill' in globals() and 'mean_em_privatized_kmack' in average_scores_yelp_gpt4_fill:
    baseline_gpt4fill_acc = average_scores_yelp_gpt4_fill['mean_em_privatized_kmack']
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='(GPT-4 Fill')
    print(f"GPT-4 Fill: {baseline_gpt4fill_acc:.3f}")
else:
    print("GPT-4 Fill baseline not found.")

# --- 4) Non-private Yelp test baseline (first 50 samples) ---
plt.axhline(y=baseline_yelp50_acc, color='green', linestyle='-.', linewidth=2,
            label='Baseline (Non-Private)')

# --- Style ---
plt.title('Accuracy vs. Epsilon')
plt.xlabel('Epsilon')
plt.ylabel('Accuracy (Exact Match)')
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend(title='Method')
plt.show()


In [ ]:
# --- load all STAMP_Polar and STAMP_Laplace datasets ---
csv_files_polar = glob.glob("/content/yelp_stamp_polar_sweep_avg_epsilon_*.csv") # Changed to yelp
csv_files_laplace = glob.glob("/content/yelp_stamp_Laplace_sweep_avg_epsilon_*.csv") # Changed to yelp

csv_files = csv_files_polar + csv_files_laplace
print("Files found by glob:", csv_files)

dataframes = {}

for file in csv_files:
    df_name = file.replace(".csv", "")
    dataframes[df_name] = pd.read_csv(file)

print("Number of dataframes loaded:", len(dataframes))

In [ ]:
# Load classifier
tokenizer = AutoTokenizer.from_pretrained("kmack/YELP-Review_Classifier")
model = AutoModelForSequenceClassification.from_pretrained("kmack/YELP-Review_Classifier")

num_rounds = 3
all_results_non_inf = []

for round_num in range(num_rounds):
    print(f"--- Starting Round {round_num + 1}/{num_rounds} ---")
    results_list = []

    for df_name, df in dataframes.items():  # Use the STAMP non-inf dataframes
        print(f"Processing DataFrame: {df_name}")

        for index, row in df.iterrows():
            question = row["question"]
            true_answers = [str(row["rating"])]  # True label as string

            privatized_context = row.get("privatized_context", "")
            original_context = row.get("original_context", "")  # <-- safely get original if available

            # --- Classify privatized context ---
            try:
                inputs = tokenizer(privatized_context, return_tensors="pt", truncation=True, padding=True)
                with torch.no_grad():
                    outputs = model(**inputs)
                pred_label = torch.argmax(outputs.logits, dim=1).item()
                answer_privatized = str(pred_label)
            except Exception as e:
                print(f"Error classifying privatized context row {index}: {e}")
                answer_privatized = ""

            # Evaluate privatized
            f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)
            cosine_privatized = calculate_cosine_similarity(answer_privatized, true_answers, model_sentence)

            # --- Store results, now with original + privatized ---
            results_list.append({
                "dataframe": df_name,
                "question": question,
                "true_answers": true_answers,
                "original_context": original_context,        # <-- added
                "privatized_context": privatized_context,    # <-- keep
                "answer_privatized": answer_privatized,
                "f1_privatized": f1_privatized,
                "em_privatized": em_privatized,
                "cosine_privatized": cosine_privatized,
            })

    # Store round results
    df_round_results = pd.DataFrame(results_list)
    all_results_non_inf.append(df_round_results)

print(f"\nFinished {num_rounds} rounds of evaluation for non-inf files.")

# Combine all rounds
df_all_results_combined_non_inf = pd.concat(all_results_non_inf, ignore_index=True)

# Save to CSV
csv_output_path_non_inf = "Yelp_evaluation_results_non_inf_STAMP_rounds.csv"
df_all_results_combined_non_inf.to_csv(csv_output_path_non_inf, index=False)

print(f"All evaluation results for non-inf files saved to {csv_output_path_non_inf}")


In [ ]:
# Load the combined evaluation results for non-inf files from CSV
csv_output_path_non_inf = "Yelp_evaluation_results_non_inf_STAMP_rounds.csv"
try:
    df_all_results_combined_non_inf = pd.read_csv(csv_output_path_non_inf)
    print(f"Successfully loaded data from {csv_output_path_non_inf}")
    display(df_all_results_combined_non_inf.head())
except FileNotFoundError:
    print(f"Error: File not found at {csv_output_path_non_inf}. Please ensure cell 23d172a1 was executed successfully.")

In [ ]:
# --- Prepare Data ---
# Extract epsilon from filename
df_all_results_combined_non_inf['epsilon'] = df_all_results_combined_non_inf['dataframe'].apply(
    lambda x: float(x.split('_')[-1])  # assumes filenames like ..._200, ..._400
)

# Add method column (Laplace or Polar)
df_all_results_combined_non_inf['method'] = df_all_results_combined_non_inf['dataframe'].apply(
    lambda x: 'STAMP Laplace' if 'laplace' in x.lower() else 'STAMP Polar'
)

# Aggregate mean accuracy by epsilon + method
df_agg_non_inf = (
    df_all_results_combined_non_inf.groupby(['epsilon', 'method'])
    .agg(mean_accuracy=('em_privatized', 'mean'))
    .reset_index()
    .sort_values('epsilon')
)

display(df_agg_non_inf)

In [ ]:
# --- Prepare Data ---
# Extract epsilon from filename
df_all_results_combined_non_inf['epsilon'] = df_all_results_combined_non_inf['dataframe'].apply(
    lambda x: float(x.split('_')[-1])  # assumes filenames like ..._200, ..._400
)

# Add method column (Laplace or Polar)
df_all_results_combined_non_inf['method'] = df_all_results_combined_non_inf['dataframe'].apply(
    lambda x: 'STAMP Laplace' if 'laplace' in x.lower() else 'STAMP Polar'
)

# Aggregate mean accuracy by epsilon + method
df_agg_non_inf = (
    df_all_results_combined_non_inf.groupby(['epsilon', 'method'])
    .agg(mean_accuracy=('em_privatized', 'mean'))
    .reset_index()
    .sort_values('epsilon')
)

# --- Plot ---
plt.figure(figsize=(8, 5))

# STAMP Polar
df_polar = df_agg_non_inf[df_agg_non_inf['method'] == 'STAMP Polar']
plt.plot(df_polar['epsilon'], df_polar['mean_accuracy'],
         label='STAMP Polar', marker='o', linestyle='-', linewidth=3) # Set linewidth

# STAMP Laplace
df_laplace = df_agg_non_inf[df_agg_non_inf['method'] == 'STAMP Laplace']
plt.plot(df_laplace['epsilon'], df_laplace['mean_accuracy'],
         label='STAMP Laplace', marker='o', linestyle='-', linewidth=3) # Set linewidth


# GPT-4 Fill + kmack baseline
# Calculate baseline directly from the loaded results CSV
if 'yelp_gpt4_fill_kmack_results' in dataframes and 'em_privatized' in dataframes['yelp_gpt4_fill_kmack_results'].columns:
    baseline_gpt4fill_acc = dataframes['yelp_gpt4_fill_kmack_results']['em_privatized'].mean()
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill)')
    print(f"Baseline (GPT-4 Fill): {baseline_gpt4fill_acc:.3f}")
else:
    print("⚠️ GPT-4 Fill baseline data not found in dataframes['yelp_gpt4_fill_kmack_results'] or 'em_privatized' column is missing.")


# --- Style ---
plt.title("Yelp: STAMP— Polar vs Laplace", fontsize=20)
plt.xlabel(r"Average per-token privacy budget $\epsilon$", fontsize=17)
plt.ylabel("Accuracy (Exact Match)", fontsize=17)
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend(title='', fontsize=15) # Removed title
plt.tick_params(axis='both', which='major', labelsize=15)
plt.show()

In [ ]:
if 'average_scores_yelp_gpt4_fill' in globals():
    display(average_scores_yelp_gpt4_fill)
else:
    print("The variable average_scores_yelp_gpt4_fill is not defined.")

In [ ]:
# --- Aggregate STAMP Polar data ---
df_stamp_polar = df_agg_non_inf[df_agg_non_inf['method'] == 'STAMP Polar']

# --- Aggregate Uniform Polar data ---
# Ensure df_all_results_combined_uniform is available and processed
if 'df_all_results_combined_uniform' not in globals():
    # Load the combined evaluation results for uniform files from CSV if not already loaded
    csv_output_path_uniform = "Yelp_evaluation_results_uniform_rounds.csv"
    try:
        df_all_results_combined_uniform = pd.read_csv(csv_output_path_uniform)
        print(f"Successfully loaded data from {csv_output_path_uniform}")
    except FileNotFoundError:
        print(f"Error: File not found at {csv_output_path_uniform}.")
        df_all_results_combined_uniform = pd.DataFrame() # Create empty dataframe to avoid errors

if not df_all_results_combined_uniform.empty:
    # Process the 'Yelp_evaluation_results_uniform_rounds' dataframe if not already processed
    if 'epsilon' not in df_all_results_combined_uniform.columns or 'method' not in df_all_results_combined_uniform.columns:
        df_all_results_combined_uniform['epsilon'] = df_all_results_combined_uniform['dataframe'].apply(
            lambda x: float(x.split('_')[-1]) if x.split('_')[-1].replace('.', '', 1).isdigit() else float('nan')
        )
        df_all_results_combined_uniform['method'] = df_all_results_combined_uniform['dataframe'].apply(
            lambda x: 'Uniform Laplace' if 'laplace' in x.lower() else 'Uniform Polar'
        )

    # Aggregate mean accuracy by epsilon + method for Uniform
    df_uniform_agg = (
        df_all_results_combined_uniform.groupby(['epsilon', 'method'])
        .agg(mean_accuracy=('em_privatized', 'mean'))
        .reset_index()
        .sort_values('epsilon')
    )
    df_uniform_polar = df_uniform_agg[df_uniform_agg['method'] == 'Uniform Polar']
else:
    df_uniform_polar = pd.DataFrame() # Create empty dataframe if uniform data not loaded

# --- Plot ---
plt.figure(figsize=(8, 5))

# STAMP Polar
if not df_stamp_polar.empty:
    plt.plot(df_stamp_polar['epsilon'], df_stamp_polar['mean_accuracy'],
             label='STAMP Polar', marker='o', linestyle='-', linewidth=3)

# Uniform Polar
if not df_uniform_polar.empty:
    plt.plot(df_uniform_polar['epsilon'], df_uniform_polar['mean_accuracy'],
             label='Uniform Polar', marker='s', linestyle='--', linewidth=3)

# Baselines
# Non-Private Yelp 50 baseline (assuming baseline_yelp50_acc is defined)
if 'baseline_yelp50_acc' in globals():
    plt.axhline(y=baseline_yelp50_acc, color='green', linestyle='-.', linewidth=2,
                label='Baseline (Non-Private Yelp 50)')
else:
     print("⚠️ Baseline (Non-Private Yelp 50) not found — please ensure it is calculated.")

# GPT-4 Fill + kmack baseline (assuming baseline_gpt4fill_acc is defined)
if 'baseline_gpt4fill_acc' in globals():
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill)')
else:
    # Calculate baseline directly from the loaded results CSV if not defined
    if 'yelp_gpt4_fill_kmack_results' in dataframes and 'em_privatized' in dataframes['yelp_gpt4_fill_kmack_results'].columns:
        baseline_gpt4fill_acc = dataframes['yelp_gpt4_fill_kmack_results']['em_privatized'].mean()
        plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                    label='Baseline (GPT-4 Fill)')
        print(f"Baseline (GPT-4 Fill): {baseline_gpt4fill_acc:.3f}")
    else:
        print("⚠️ GPT-4 Fill baseline data not found.")


# --- Style ---
plt.title("Yelp: Polar— Uniform vs STAMP", fontsize=20) # Consistent title
plt.xlabel(r"Average per-token privacy budget $\epsilon$", fontsize=17)
plt.ylabel("Accuracy (Exact Match)", fontsize=17)
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend(title='', fontsize=15) # Removed title
plt.tick_params(axis='both', which='major', labelsize=15)
plt.show()

In [ ]:
# --- load all uniform_Polar and uniform_Laplace datasets ---
csv_files_polar = glob.glob("/content/yelp_uniform_polar_sweep_avg_epsilon_*.csv") # Changed to yelp
csv_files_laplace = glob.glob("/content/yelp_uniform_laplace_sweep_avg_epsilon_*.csv") # Changed to yelp

csv_files = csv_files_polar + csv_files_laplace
print("Files found by glob:", csv_files)

dataframes = {}

for file in csv_files:
    df_name = file.replace(".csv", "")
    dataframes[df_name] = pd.read_csv(file)

print("Number of dataframes loaded:", len(dataframes))

In [ ]:
# Load classifier
tokenizer = AutoTokenizer.from_pretrained("kmack/YELP-Review_Classifier")
model = AutoModelForSequenceClassification.from_pretrained("kmack/YELP-Review_Classifier")

num_rounds = 3
all_results_uniform = []

for round_num in range(num_rounds):
    print(f"--- Starting Round {round_num + 1}/{num_rounds} ---")
    results_list = []

    for df_name, df in dataframes.items():  # Uniform Laplace/Polar
        print(f"Processing DataFrame: {df_name}")

        for index, row in df.iterrows():
            question = row["question"]
            true_answers = [str(row["rating"])]  # Gold label

            privatized_context = row.get("privatized_context", "")
            original_context = row.get("original_context", "")

            # --- Classify privatized context ---
            try:
                inputs = tokenizer(privatized_context, return_tensors="pt", truncation=True, padding=True)
                with torch.no_grad():
                    outputs = model(**inputs)
                pred_label = torch.argmax(outputs.logits, dim=1).item()
                answer_privatized = str(pred_label)
            except Exception as e:
                print(f"Error classifying privatized context row {index}: {e}")
                answer_privatized = ""

            # Evaluate privatized
            f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)
            cosine_privatized = calculate_cosine_similarity(answer_privatized, true_answers, model_sentence)

            # --- Save results with both contexts ---
            results_list.append({
                "dataframe": df_name,
                "question": question,
                "true_answers": true_answers,
                "original_context": original_context,
                "privatized_context": privatized_context,
                "answer_privatized": answer_privatized,
                "f1_privatized": f1_privatized,
                "em_privatized": em_privatized,
                "cosine_privatized": cosine_privatized,
            })

    # Store round results
    df_round_results = pd.DataFrame(results_list)
    all_results_uniform.append(df_round_results)

print(f"\nFinished {num_rounds} rounds of evaluation for uniform datasets.")

# Combine all results
df_all_results_combined_uniform = pd.concat(all_results_uniform, ignore_index=True)

# Save to CSV
csv_output_path_uniform = "Yelp_evaluation_results_uniform_rounds.csv"
df_all_results_combined_uniform.to_csv(csv_output_path_uniform, index=False)

print(f"All evaluation results for uniform files saved to {csv_output_path_uniform}")


In [ ]:
# --- Aggregate STAMP ---
df_all_results_combined_non_inf['epsilon'] = df_all_results_combined_non_inf['dataframe'].apply(
    lambda x: float(x.split('_')[-1]) if x.split('_')[-1].replace('.', '', 1).isdigit() else float('nan')
)
df_all_results_combined_non_inf['method'] = df_all_results_combined_non_inf['dataframe'].apply(
    lambda x: 'STAMP Laplace' if 'laplace' in x.lower() else 'STAMP Polar'
)
df_stamp_agg = (
    df_all_results_combined_non_inf.groupby(['epsilon', 'method'])
    .agg(mean_accuracy=('em_privatized', 'mean'))
    .reset_index()
)

# --- Aggregate UNIFORM ---
df_all_results_combined_uniform['epsilon'] = df_all_results_combined_uniform['dataframe'].apply(
    lambda x: float(x.split('_')[-1]) if x.split('_')[-1].replace('.', '', 1).isdigit() else float('nan')
)
df_all_results_combined_uniform['method'] = df_all_results_combined_uniform['dataframe'].apply(
    lambda x: 'Uniform Laplace' if 'laplace' in x.lower() else 'Uniform Polar'
)
df_uniform_agg = (
    df_all_results_combined_uniform.groupby(['epsilon', 'method'])
    .agg(mean_accuracy=('em_privatized', 'mean'))
    .reset_index()
)


In [ ]:
plt.figure(figsize=(12, 7))

# STAMP Polar
df_stamp_polar = df_stamp_agg[df_stamp_agg['method'] == 'STAMP Polar']
plt.plot(df_stamp_polar['epsilon'], df_stamp_polar['mean_accuracy'],
         label='STAMP Polar', marker='o', linestyle='-')

# Uniform Polar
df_uniform_polar = df_uniform_agg[df_uniform_agg['method'] == 'Uniform Polar']
plt.plot(df_uniform_polar['epsilon'], df_uniform_polar['mean_accuracy'],
         label='Uniform Polar', marker='s', linestyle='--')

# Baselines
plt.axhline(y=baseline_yelp50_acc, color='green', linestyle='-.', linewidth=2,
            label='Baseline (Non-Private Yelp 50)')
if 'average_scores_yelp_gpt4_fill' in globals() and 'mean_em_privatized_kmack' in average_scores_yelp_gpt4_fill:
    baseline_gpt4fill_acc = average_scores_yelp_gpt4_fill['mean_em_privatized_kmack']
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill + kmack)')

plt.title("Polar: Uniform vs STAMP")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy (Exact Match)")
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12, 7))

# STAMP Polar
plt.plot(df_stamp_polar['epsilon'], df_stamp_polar['mean_accuracy'],
         label='STAMP Polar', marker='o', linestyle='-')

# STAMP Laplace
df_stamp_laplace = df_stamp_agg[df_stamp_agg['method'] == 'STAMP Laplace']
plt.plot(df_stamp_laplace['epsilon'], df_stamp_laplace['mean_accuracy'],
         label='STAMP Laplace', marker='^', linestyle='--')

# Baselines
plt.axhline(y=baseline_yelp50_acc, color='green', linestyle='-.', linewidth=2,
            label='Baseline (Non-Private Yelp 50)')
if 'average_scores_yelp_gpt4_fill' in globals() and 'mean_em_privatized_kmack' in average_scores_yelp_gpt4_fill:
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill + kmack)')

plt.title("STAMP: Polar vs Laplace")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy (Exact Match)")
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# --- Step 3: Plot Uniform Polar vs Uniform Laplace ---
plt.figure(figsize=(12, 7))

# Uniform Polar
df_uniform_polar = df_uniform_agg[df_uniform_agg['method'] == 'Uniform Polar']
plt.plot(df_uniform_polar['epsilon'], df_uniform_polar['mean_accuracy'],
         label='Uniform Polar', marker='o', linestyle='-')

# Uniform Laplace
df_uniform_laplace = df_uniform_agg[df_uniform_agg['method'] == 'Uniform Laplace']
plt.plot(df_uniform_laplace['epsilon'], df_uniform_laplace['mean_accuracy'],
         label='Uniform Laplace', marker='^', linestyle='--')

# Baselines
plt.axhline(y=baseline_yelp50_acc, color='green', linestyle='-.', linewidth=2,
            label='Baseline (Non-Private Yelp 50)')
if 'average_scores_yelp_gpt4_fill' in globals() and 'mean_em_privatized_kmack' in average_scores_yelp_gpt4_fill:
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill + kmack)')

# Style
plt.title("Uniform: Polar vs Laplace")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy (Exact Match)")
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(14, 8))

# --- STAMP ---
df_stamp_polar = df_stamp_agg[df_stamp_agg['method'] == 'STAMP Polar']
plt.plot(df_stamp_polar['epsilon'], df_stamp_polar['mean_accuracy'],
         label='STAMP Polar', marker='o', linestyle='-')

df_stamp_laplace = df_stamp_agg[df_stamp_agg['method'] == 'STAMP Laplace']
plt.plot(df_stamp_laplace['epsilon'], df_stamp_laplace['mean_accuracy'],
         label='STAMP Laplace', marker='^', linestyle='--')

# --- UNIFORM ---
df_uniform_polar = df_uniform_agg[df_uniform_agg['method'] == 'Uniform Polar']
plt.plot(df_uniform_polar['epsilon'], df_uniform_polar['mean_accuracy'],
         label='Uniform Polar', marker='s', linestyle='-.')

df_uniform_laplace = df_uniform_agg[df_uniform_agg['method'] == 'Uniform Laplace']
plt.plot(df_uniform_laplace['epsilon'], df_uniform_laplace['mean_accuracy'],
         label='Uniform Laplace', marker='d', linestyle=':')

# --- INF ---
df_inf_results['epsilon'] = df_inf_results['dataframe'].apply(
    lambda x: float(x.split('_')[-1].replace('.csv','')) if 'inf' not in x.lower() else float('inf')
)
df_inf_results['method'] = df_inf_results['dataframe'].apply(
    lambda x: 'Inf Laplace' if 'laplace' in x.lower() else 'Inf Polar'
)
df_inf_agg = (
    df_inf_results.groupby(['epsilon','method'])
    .agg(mean_accuracy=('em_privatized','mean'))
    .reset_index()
)

df_inf_polar = df_inf_agg[df_inf_agg['method'] == 'Inf Polar']
plt.plot(df_inf_polar['epsilon'], df_inf_polar['mean_accuracy'],
         label='Inf Polar', marker='x', linestyle='-')

df_inf_laplace = df_inf_agg[df_inf_agg['method'] == 'Inf Laplace']
plt.plot(df_inf_laplace['epsilon'], df_inf_laplace['mean_accuracy'],
         label='Inf Laplace', marker='x', linestyle='--')

# --- Baselines ---
plt.axhline(y=baseline_yelp50_acc, color='green', linestyle='-.', linewidth=2,
            label='Baseline (Non-Private Yelp 50)')
if 'average_scores_yelp_gpt4_fill' in globals() and 'mean_em_privatized_kmack' in average_scores_yelp_gpt4_fill:
    plt.axhline(y=baseline_gpt4fill_acc, color='red', linestyle=':', linewidth=2,
                label='Baseline (GPT-4 Fill + kmack)')

# --- Style ---
plt.title("Comparison of All Mechanisms (Polar vs Laplace, STAMP vs Uniform vs Inf)")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy (Exact Match)")
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend(title="Mechanism", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
